<a href="https://colab.research.google.com/github/lorenzcipher/Hackathon-Junction2021/blob/main/Junction2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PACKAGES

In [43]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import  preprocessing

# **Lecture de la data**

In [44]:
#load data train
df_train = pd.read_csv("train.csv", sep=",", encoding="utf-8")

# **Demande d'email pour valeur Null**

In [45]:
df_train.replace('0113eba6-6928-461a-b994-35a0b2eb9f4e', np.nan)

,case_id,equipment_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
0,b7c775ad-4ebe-4848-9c53-37e7c5658e21,ele0000754,2018-10-03,ar00000174,art01,arc03,ga00000803,ut012,7,6,8,tp006,1
1,b177eefd-3946-4949-9699-0a91879350f9,ele0000789,2018-10-03,ar00000248,art02,arc03,ga00000269,ut005,4,6,2,tp001,1
2,fc2d568a-c53c-43ef-8871-a49ec22ab3b1,ele0001227,2018-10-03,ar00000158,art02,arc03,ga00000064,ut011,6,6,7,tp006,1
3,8e645922-1268-4c6b-ae6b-7b1605689cca,ele0001754,2018-10-03,ar00000105,art01,arc03,ga00000662,ut005,8,7,8,tp014,1
4,b44f10b1-9238-44db-8f0c-2d68e8c015a7,ele0002087,2018-10-03,ar00000148,art02,arc03,ga00001256,ut011,7,6,8,tp013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115470,8eda4489-c34c-4e8a-b592-ad57b5fd842a,ele0029759,2020-01-30,ar00000124,art01,arc03,ga00000817,ut011,6,7,8,tp005,1
115471,NaN,ele0029783,2020-01-30,ar00000291,art02,arc01,ga00001027,ut001,4,3,1,tp015,1
115472,c2eab0dc-218a-4a95-ab28-47449f42f660,ele0029785,2020-01-30,ar00000048,art01,arc02,ga00000355,ut011,2,1,4,tp002,1
115473,c2eab0dc-218a-4a95-ab28-47449f42f660,ele0029785,2020-01-30,ar00000293,art01,arc02,ga00000355,ut011,2,1,4,tp002,1


# Dimension de la data


In [46]:
df_train.shape

(115475, 13)

#Type de chaque attribut


In [47]:
df_train.dtypes

case_id                           object
equipment_id                      object
completion_date                   object
action_recommendation_id          object
action_recommendation_type        object
action_recommendation_category    object
equipment_area                    object
usage_type                        object
speed_category                     int64
load_category                      int64
floors_category                    int64
equipment_category                object
feedback                           int64
dtype: object

# Verification si y a des valeurs manquates

In [48]:
(df_train.isna().sum() / df_train.shape[0]).sort_values()

case_id                           0.0
equipment_id                      0.0
completion_date                   0.0
action_recommendation_id          0.0
action_recommendation_type        0.0
action_recommendation_category    0.0
equipment_area                    0.0
usage_type                        0.0
speed_category                    0.0
load_category                     0.0
floors_category                   0.0
equipment_category                0.0
feedback                          0.0
dtype: float64

# Supprimer les variables non significatif 

In [49]:
df_train.drop(['equipment_id', 'case_id', 'action_recommendation_id', 'equipment_area','completion_date'], axis=1, inplace=True)
df_train

,action_recommendation_type,action_recommendation_category,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
0,art01,arc03,ut012,7,6,8,tp006,1
1,art02,arc03,ut005,4,6,2,tp001,1
2,art02,arc03,ut011,6,6,7,tp006,1
3,art01,arc03,ut005,8,7,8,tp014,1
4,art02,arc03,ut011,7,6,8,tp013,1
...,...,...,...,...,...,...,...,...
115470,art01,arc03,ut011,6,7,8,tp005,1
115471,art02,arc01,ut001,4,3,1,tp015,1
115472,art01,arc02,ut011,2,1,4,tp002,1
115473,art01,arc02,ut011,2,1,4,tp002,1


# Stat Descriptif

#Encodage OneHotEncoder

In [50]:
quali_df = df_train[['action_recommendation_type', 'action_recommendation_category', 'usage_type', 'equipment_category']]

enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc.fit(quali_df)
onehotlabels = enc.transform(quali_df).toarray()
onehotlabels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [51]:
quanti_vars = df_train[["speed_category", "load_category", "floors_category"]].to_numpy()
new_df = np.concatenate((onehotlabels, quanti_vars), axis=1)

new_df

array([[1., 0., 0., ..., 7., 6., 8.],
       [0., 1., 0., ..., 4., 6., 2.],
       [0., 1., 0., ..., 6., 6., 7.],
       ...,
       [1., 0., 0., ..., 2., 1., 4.],
       [1., 0., 0., ..., 2., 1., 4.],
       [1., 0., 0., ..., 6., 4., 8.]])

# Data split train/validation

In [52]:
X_train, X_val, y_train, y_val = train_test_split(new_df, df_train["feedback"], test_size=0.2, random_state=42, stratify=df_train["feedback"])

In [53]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

In [54]:
#Train the model using the training sets
clf.fit(X_train, y_train)

SVC(kernel='linear')

#Predict the response for test dataset

In [ ]:
y_pred = clf.predict(X_val)


In [ ]:
precision =metrics.precision_score(y_val, y_pred,average='weighted')
precision

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7703206926819067

In [ ]:
recall =metrics.recall_score(y_val, y_pred,average='weighted')
recall

0.877679151331457

In [ ]:
F2 =metrics.fbeta_score(y_val, y_pred, beta=2)
F2

0.9728821694264459

#Random Forest

Create a Gaussian Classifier


In [ ]:
clf_rf = RandomForestClassifier(n_estimators=100)

Train the model using the training sets y_pred=clf.predict(X_test)


In [ ]:
clf_rf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred_rf=clf_rf.predict(X_val)

In [ ]:
precision_rf =metrics.precision_score(y_val, y_pred_rf,average='weighted')
precision_rf

0.8617840326944726

In [ ]:
recall_rf =metrics.recall_score(y_val, y_pred_rf,average='weighted')
recall_rf

0.8858627408529984

In [ ]:
F2_rf =metrics.fbeta_score(y_val, y_pred_rf, beta=2)
F2_rf

0.962705227858652

Model Accuracy, how often is the classifier correct?

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_val, y_pred))

Accuracy: 0.8841307642346828


#Logistic Regression

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
y_pred_reg=logreg.predict(X_val)

In [ ]:
precision_reg =metrics.precision_score(y_val, y_pred_reg,average='weighted')
recall_reg =metrics.recall_score(y_val, y_pred_reg,average='weighted')
F2_reg =metrics.fbeta_score(y_val, y_pred_reg, beta=2)
F2_reg


0.9725502621118728

#TEST

In [ ]:
#load data train
df_test = pd.read_csv("test.csv", sep=",", encoding="utf-8")
df_copy = df_test.copy()

df_test.replace('62bf183c-eb55-4da1-b654-f245b320c6c0', np.nan)



df_test.drop(['equipment_id', 'case_id', 'action_recommendation_id', 'equipment_area','completion_date'], axis=1, inplace=True)


quali_df_test = df_test[['action_recommendation_type', 'action_recommendation_category', 'usage_type', 'equipment_category']]


enc_test = preprocessing.OneHotEncoder(handle_unknown='ignore')


enc_test.fit(quali_df_test)


onehotlabels_test = enc.transform(quali_df_test).toarray()


quanti_vars_test = df_test[["speed_category", "load_category", "floors_category"]].to_numpy()

new_df_test = np.concatenate((onehotlabels_test, quanti_vars_test), axis=1)


y_pred_test = clf.predict(new_df_test)

submit= df_copy[['case_id','action_recommendation_id']].to_numpy()

submit=pd.DataFrame(submit,columns=['case_id','action_recommendation_id'])


submit.insert(2, "feedback", y_pred_test, allow_duplicates=False)

submit.to_csv('/content/submit_final.csv', index=False)



#Test

Eliminer les colonnes manquantes 

Calcul des feedback positif et négatif

In [ ]:
X_2['feedback'].value_counts(normalize=True)

1    0.877688
0    0.122312
Name: feedback, dtype: float64

In [ ]:
X_2.dtypes

case_id                           int64
completion_date                   int64
action_recommendation_id          int64
action_recommendation_type        int64
action_recommendation_category    int64
equipment_area                    int64
usage_type                        int64
speed_category                    int64
load_category                     int64
floors_category                   int64
equipment_category                int64
feedback                          int64
dtype: object

STATISTIQUE DESCRIPTIF

In [ ]:
X_2.describe()

AttributeError: ignored

Class Distribution

In [ ]:
X_2.groupby('action_recommendation_category').size()

action_recommendation_category
0    10193
1    25727
2    75955
3      827
4     1493
5     1280
dtype: int64

Correlation entre les attributs 

In [ ]:
X_2.corr(method='pearson')

,case_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
case_id,1.000000,-0.209332,0.084480,-0.349478,-0.046613,0.020520,-0.014881,-0.020279,-0.033196,-0.003272,0.000087,-0.042681
completion_date,-0.209332,1.000000,-0.087344,0.049133,-0.020876,-0.019109,0.031730,-0.034938,-0.030409,-0.021983,-0.022338,-0.015378
action_recommendation_id,0.084480,-0.087344,1.000000,-0.230223,-0.007881,-0.009275,-0.018359,-0.113613,-0.109110,-0.079376,-0.052588,-0.040008
action_recommendation_type,-0.349478,0.049133,-0.230223,1.000000,0.032518,-0.014606,0.000021,-0.013916,0.047011,-0.063425,-0.035604,0.101247
action_recommendation_category,-0.046613,-0.020876,-0.007881,0.032518,1.000000,-0.027535,-0.039235,0.323928,0.276001,0.291458,0.034720,0.087621
equipment_area,0.020520,-0.019109,-0.009275,-0.014606,-0.027535,1.000000,-0.004169,0.002108,0.026787,-0.017778,-0.004496,-0.015219
usage_type,-0.014881,0.031730,-0.018359,0.000021,-0.039235,-0.004169,1.000000,-0.082237,-0.216286,0.097932,0.007878,0.005777
speed_category,-0.020279,-0.034938,-0.113613,-0.013916,0.323928,0.002108,-0.082237,1.000000,0.434765,0.688064,0.144709,0.089443
load_category,-0.033196,-0.030409,-0.109110,0.047011,0.276001,0.026787,-0.216286,0.434765,1.000000,0.190428,0.186855,0.105908
floors_category,-0.003272,-0.021983,-0.079376,-0.063425,0.291458,-0.017778,0.097932,0.688064,0.190428,1.000000,0.142209,0.055462


Skewness

In [ ]:
X_2.skew()

case_id                           0.232542
completion_date                  -0.304804
action_recommendation_id         -0.002131
action_recommendation_type        0.957824
action_recommendation_category    0.286227
equipment_area                    0.073642
usage_type                       -1.029188
speed_category                    0.060989
load_category                    -0.406180
floors_category                  -0.404027
equipment_category                1.288622
feedback                         -2.305492
dtype: float64

In [ ]:
#applique le filtre dépend le feedback pour générer deux classes
# 1 = Oui pour venir sur place 
# 0= Non pas la peine
yes_train = X_train[X_train['feedback']==1][0:200]
no_train = X_train[X_train['feedback']==0][0:200]
#visualisation des données par rapport deux attributs
axes =yes_train.plot(kind='scatter',x='action_recommendation_type',y='action_recommendation_category',color='blue',label='Yes comparision')
no_train.plot(kind='scatter',x='action_recommendation_type',y='action_recommendation_category',color='red',label='No comparision',ax=axes)